In [4]:
# This program uses Policy Iteration to find the the optimal policy.
# in a Markov Decision Process. Value Iteration on MDP is a GREEDY Algorithm (It will take the highest reward at each step).
# Note: This program is specific to the Markov Decision Process as described below. For more states or actions, the code must be adjusted to
# incorporate Action A, Action B, ..., Action N. For different states,
import pandas as pd

# transition_probabilities_A = transition probabilities for Action A, transition_probabilities_B = transition probabilities for Action B
# transition_probabilities_A = [s1[s1, s2, s3], s2[s1, s2, s3], s3[s1, s2, s3]]
# transition_probabilities_B = [s1[s1, s2, s3], s2[s1, s2, s3], s3[s1, s2, s3]]
# [s1, s2, s3] original state rewards


def get_transition_probabilities_and_rewards_from_csv(transitionFile_A, transitionFile_B, rewardFile):
  # Read in CSV files
  transition_A_df = pd.read_csv(transitionFile_A, header=None)
  transition_B_df = pd.read_csv(transitionFile_B, header=None)
  rewards_df = pd.read_csv(rewardFile, header=None)

  # Process Markov State Transition Probabilities, ACTION A
  transition_probabilities_A = []
  for i in range(len(transition_A_df)):
    temp = []
    ser = pd.Series(transition_A_df.iloc[i])
    for j in range(transition_A_df.shape[0]):
      temp.append(ser.iloc[j])
    transition_probabilities_A.append(temp)

  # Process Markov State Transition Probabilities, ACTION B
  transition_probabilities_B = []
  for i in range(len(transition_B_df)):
    temp = []
    ser = pd.Series(transition_B_df.iloc[i])
    for j in range(transition_B_df.shape[0]):
      temp.append(ser.iloc[j])
    transition_probabilities_B.append(temp)

  # Process Markov State Rewards
  state_reward = []
  for i in range(len(rewards_df)):
    ser = pd.Series(rewards_df.iloc[i])
    state_reward.append(ser.iloc[0])

  return transition_probabilities_A, transition_probabilities_B, state_reward


# Given a particular state S_i, calculate the J*(S_i) Reward
def J_star_reward(state, original_state_rewards, prior_state_rewards, gamma, transition_prob_A, transition_prob_B, policyAction='A'):
  reward = original_state_rewards[state]
  Jsum = 0
  for i in range(len(prior_state_rewards)):
    if policyAction == 'A':
      Jsum += transition_prob_A[state][i] * prior_state_rewards[i]  # Calculate a Jreward for the action.
    else:
      Jsum += transition_prob_B[state][i] * prior_state_rewards[i]

  reward +=  gamma * Jsum

  return reward


# Given a particular state S_i, calculate the J*(S_i) Reward
def policy_improvement_J_star_reward(state, original_state_rewards, prior_state_rewards, gamma, transition_prob_A, transition_prob_B):
  reward = original_state_rewards[state]
  Jsum_A = 0
  Jsum_B = 0
  for i in range(len(prior_state_rewards)):
      Jsum_A += transition_prob_A[state][i] * prior_state_rewards[i]  # Calculate a Jreward for each action, and take the MAX for most reward gain.
      Jsum_B += transition_prob_B[state][i] * prior_state_rewards[i]

  reward_A = reward + gamma * Jsum_A
  reward_B = reward + gamma * Jsum_B

  if reward_A > reward_B:
    return 'A'
  else:
    return 'B'


# Given an Epsilon threshold, calculate the Markov System Value Iteration Reward until the difference between consecutive
# rewards is smaller than Epsilon.
def MDP_Policy_Iteration_Helper(Epsilon, discountFactor, transitionProbabilities_A, transition_probabilities_B, stateRewards, est_policy):
  # At K=1 iteration, the state reward is the reward
  discounted_rewards = stateRewards

  while True:
      # Iterate through every State and calculate the discounted rewards against epsilon
      temp = []  # temp holds the value of the current iteration-K rewards.
      for i in range(len(discounted_rewards)):
          temp.append(J_star_reward(i, stateRewards, discounted_rewards, discountFactor, transitionProbabilities_A, transition_probabilities_B, est_policy[i]))

      # Find the Max difference between K and K-1 iterations of Rewards, and compare it to Epsilon.
      # If the difference is smaller than Epsilon, stop calculating. The discounted rewards have converged.
      Max_diff = 0
      for i in range(len(temp)):
          curr_diff = abs(temp[i] - discounted_rewards[i])
          Max_diff = curr_diff if curr_diff > Max_diff else Max_diff
      # Check to see if Error threshold has been reached for max difference between subsequent k-iterations. If so, the rewards have converged.
      # Return Long Term Discounted Rewards and the Policy.
      # Round the calculated Discounted Rewards to 2 decimal places.
      if Max_diff < Epsilon:
          longTemDiscountedRewards = [round(elem, 2) for elem in temp]
          return longTemDiscountedRewards

      discounted_rewards = temp


def Markov_Decision_Process_Policy_Iteration(Epsilon, discountFactor, transProb_A, transProb_B, stateRewards, initPolicy):
  initialPolicy = initPolicy   # Policy to feed into the Policy Iteration task.

  while True:
    # (1) Policy Evaluation: Calculate the Long Term Discounted Rewards of the Markov Decision Problem
    long_term_discounted_rewards = MDP_Policy_Iteration_Helper(Epsilon, discountFactor, transProb_A, transProb_B, stateRewards, initialPolicy)
    # (2) Policy Improvement
    updated_policy = []
    for i in range(len(long_term_discounted_rewards)):
      updated_policy.append(policy_improvement_J_star_reward(i, stateRewards, long_term_discounted_rewards, discountFactor, transProb_A, transProb_B))

    print(initialPolicy)
    # Check to see if policy is changed. If changed, continue to iterate. It unchanged, the Optimal Policy has been found; return findings.
    if initialPolicy == updated_policy:
      return updated_policy
    else:
      initialPolicy = updated_policy




# Read in the MDP information, consisting of transition probabilities per ACTION (A, B in this exampe MDP)and state rewards
epsilon = 0.001
discount_factor = 0.9
initial_policy = ['A', 'A', 'A']
markovTransitionFileName_A = '/content/drive/MyDrive/MDP_A Transition Probabilities - Sheet1.csv'
markovTransitionFileName_B = '/content/drive/MyDrive/MDP_B Transition Probabilities - Sheet1.csv'
markovRewardFileName = '/content/drive/MyDrive/MDP State Rewards - Sheet1.csv'
trans_prob_A, trans_prob_B, st_rewards = get_transition_probabilities_and_rewards_from_csv(markovTransitionFileName_A, markovTransitionFileName_B, markovRewardFileName)

# Run Policy Iteration
optimal_policy = Markov_Decision_Process_Policy_Iteration(epsilon, discount_factor, trans_prob_A, trans_prob_B, st_rewards, initial_policy)
print(f"The Optimal Policy is: {optimal_policy} for states [S1, S2, S3], respectively.")




['A', 'A', 'A']
['B', 'B', 'A']
The Optimal Policy is: ['B', 'B', 'A'] for states [S1, S2, S3], respectively.
